<a href="https://colab.research.google.com/github/Bebe-Ai/Ai-Playground/blob/main/week3_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers datasets

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
import random


In [2]:
# Example auto-generated Q&A
questions = [
    "What is the delivery time?",
    "Do you ship internationally?",
    "How can I return a product?",
    "What payment methods do you accept?",
    "Is there a warranty?"
]

answers = [
    "Delivery usually takes 3-5 business days.",
    "Yes, we ship worldwide.",
    "You can return any product within 30 days.",
    "We accept credit cards, PayPal, and Apple Pay.",
    "All products have a 1-year warranty."
]

# Generate 500 fake examples
data = []
for i in range(500):
    q = random.choice(questions)
    a = answers[questions.index(q)]
    data.append({"question": q, "answer": a})

df = pd.DataFrame(data)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split train/test
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

print(train_dataset[0])


{'question': 'Is there a warranty?', 'answer': 'All products have a 1-year warranty.'}


In [3]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # T5 is great for Q&A

def tokenize(batch):
    # Prefix “question: ” is common for T5-style tasks
    inputs = ["question: " + q for q in batch["question"]]
    targets = batch["answer"]
    model_inputs = tokenizer(inputs, padding=True, truncation=True, max_length=64)
    labels = tokenizer(targets, padding=True, truncation=True, max_length=64)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


In [5]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[]  # This disables W&B (and any other reporting)
)


In [6]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [7]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,No log,0.303243
2,No log,0.022510
3,No log,0.009168


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=300, training_loss=1.0846614583333334, metrics={'train_runtime': 327.9227, 'train_samples_per_second': 3.659, 'train_steps_per_second': 0.915, 'total_flos': 3489280819200.0, 'train_loss': 1.0846614583333334, 'epoch': 3.0})

In [8]:
def answer_question(question):
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test examples
print(answer_question("Do you ship internationally?"))
print(answer_question("How can I return a product?"))


Yes, we ship worldwide.
You can return any product within 30 days.


In [9]:
model.save_pretrained("./customer_support_model")
tokenizer.save_pretrained("./customer_support_model")


('./customer_support_model/tokenizer_config.json',
 './customer_support_model/special_tokens_map.json',
 './customer_support_model/spiece.model',
 './customer_support_model/added_tokens.json',
 './customer_support_model/tokenizer.json')

In [12]:
def answer_question(question):
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test examples
print(answer_question("How long does the return time take?"))
print(answer_question("What type o products do you have?"))


We take 3-5 business days.
We have a variety of products.


In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load your trained model
tokenizer = AutoTokenizer.from_pretrained("./customer_support_model")
model = AutoModelForSeq2SeqLM.from_pretrained("./customer_support_model")

# Function to ask the bot a question
def ask_bot(question):
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=80)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Interactive loop
print("💬 Customer Support Bot is ready! Type 'quit' to stop.\n")

while True:
    question = input("You: ")
    if question.lower() == "quit":
        print("Bot: Goodbye 👋")
        break
    answer = ask_bot(question)
    print("Bot:", answer)


💬 Customer Support Bot is ready! Type 'quit' to stop.

You: Do you have a shipping option to Hong Kong?
Bot: Yes, we have a shipping option to Hong Kong.
You: Do you sell bags?
Bot: Yes, we sell bags.
You: What type of bags
Bot: bags, bags, and other types of bags.
You: Haha you are unny
Bot: Haha, you are unneel.
You: Tell me something about the products.
Bot: Tell me something about the products.
You: quit
Bot: Goodbye 👋
